In [5]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Updating registry at `C:\Users\nsthtz\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...


In [1]:
using DelimitedFiles
data = readdlm(joinpath("dummyMajor.csv"),';')
#data = readdlm(joinpath("dummyMinor.csv"),',')
data = round.(Int, data)

128×24 Array{Int64,2}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 ⋮              ⋮              ⋮         

# # Convert piano roll into Tx12 matrix


In [2]:
#

timestep_intervals = zeros(Int8, size(data, 2), 12)
i=1
data[:, 1]

for i in 1:size(data, 2)
    active_notes = findall(x->x==1, data[:,i])
    #println(active_notes)
    for note in active_notes
        timestep_intervals[i, note%12+1] = 1
    end
    #println(timestep_chords[i,:])
end

timestep_intervals

24×12 Array{Int8,2}:
 0  0  0  0  0  0  0  0  0  0  1  0
 1  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  0  1  0

# # Pattern matcher to identify the best-fit key and scale from existing notes

In [3]:
# Functions and Global

pattern = [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1]

# Weight of core notes (1, 4, 5) doubled
majorpattern = [1, 0, 0.5, 0, 0.5, 1, 0, 1, 0, 0.5, 0, 0.5]
minorpattern = [0.5, 0, 1, 0, 1, 0.5, 0, 0.5, 0, 1, 0, 0.5]

Keys = Dict{Integer, String}(1 => "C", 2 => "C#", 3 => "D", 4 => "D#", 5 => "E", 6 => "F", 7 => "F#", 8 => "G", 9 => "G#", 10 => "A", 11 => "A#", 12 => "B")


#Score In: Tx12 Matrix, pattern. Out: Best position(s)
function score_key(matrix, pattern)
    counter = zeros(Int8, 1, 12)
    for i in 1:size(matrix, 2)
        counter[i] = sum(matrix[:,i])
    end
    println("Key Counter is ", counter)
    
    key_score = zeros(Int8, 12, 1)
    
    for i in 1:size(pattern, 1)
        for j in 1:size(pattern, 1)
            key_score[i] += circshift(pattern, i-1)[j]*counter[j]
        end
    end
    println("Key Score is ", key_score)
    return key_score
end

# Find maximum score from list
function findmaximum(score_result)
    positions = []
    max_value = maximum(score_result)
    for (i, x) in enumerate(score_result)
        if x == max_value
            push!(positions, i)
        end
    end
    #positions = [i for (i, x) in enumerate(score_result) if x == max_value]
    return positions
end

# Find an optimal scale
function score_tonality(matrix, positions)
    
    counter = zeros(Int8, 1, 12)
        for i in 1:size(matrix, 2)
            counter[i] = sum(matrix[:,i])
        end
    
    tonality_scores = []
    
    for position in positions
        counter_shifted = circshift(counter, (0,-(position-1)))
        println("Position Counter for ", position, " is ", counter_shifted)
        
        major_score = 0
        minor_score = 0
        
        for i in 1:size(counter_shifted, 2)
            major_score += majorpattern[i]*counter_shifted[i]
            minor_score += minorpattern[i]*counter_shifted[i]
        end

        if major_score > minor_score
            push!(tonality_scores, [position, major_score, 1])
        else
            push!(tonality_scores, [position, minor_score, 0])
        end
        
    end 
    
    return tonality_scores
end

score_tonality (generic function with 1 method)

In [4]:
# Score matrix after pattern
key_score_result = score_key(timestep_intervals, pattern)
# score_result = [9, 5, 18, 9, 23, 18, 8, 2, 7, 16, 12, 11]

# Find positions of maximum score
positions = findmaximum(key_score_result)
sort!(positions)

# # Use positions to find the best key

tonality_results = score_tonality(timestep_intervals, positions)

# [Position (1 = C), Score, Scale (1 = Major, 0 = Minor)]
best_key = [0, 0, 1]

# In case of tied scores, check each for the best scale
for result_group in tonality_results
    if best_key[2] < result_group[2]
        best_key[1] = result_group[1]
        best_key[2] = result_group[2]
        best_key[3] = result_group[3]
    end
end
    
#Save Key
if best_key[3] == 1
    key = best_key[1]
    key_print = Keys[best_key[1]]
    scale = "Major"
else
    key = (best_key[1]-3)%12
    key_print = Keys[((best_key[1]-3)%12)]
    scale = "Minor"
end
    
println(key, " ", key_print, " ", scale)

Key Counter is Int8[2 0 2 5 0 3 0 3 0 3 6 0]
Key Score is Int8[13; 16; 8; 21; 8; 19; 14; 10; 19; 5; 24; 11]
Position Counter for 11 is Int8[6 0 2 0 2 5 0 3 0 3 0 3]
11 A# Major


## Define patterns for chord functions relative to key

In [5]:
# Function entry: "Name" => ([notes], [Bass note], [Double/Triple notes], [Rhythmical Weight (All, Strong, Weak, Off)], [Chord Specific rules])

functions = Dict{String, Array{Int64, 1}}

if scale == "Major"
#Major  =  [1, 0, 2, 0, 3, 4, 0, 5, 0, 6, 0, 7]
#
    
    functions = Dict( "T"  =>  (circshift([1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ key ], [ key ], ["A"]),
    "S"    =>  (circshift([1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], (key-1)), [ ((5+(key-1))%12)+1 ], [ ((5+(key-1))%12)+1 ], ["A"]),
    "D"    =>  (circshift([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1], (key-1)), [ ((7+(key-1))%12)+1 ], [ ((7+(key-1))%12)+1 ], ["A"]),
    "T-5"  =>  (circshift([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], (key-1)), [ key ], [ key ], ["A"]),
    "S-5"  =>  (circshift([0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], (key-1)), [ ((5+(key-1))%12)+1 ], [ ((5+(key-1))%12)+1 ], ["A"]),
    "D-5"  =>  (circshift([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1], (key-1)), [ ((7+(key-1))%12)+1 ], [ ((7+(key-1))%12)+1 ], ["A"]),
    "T/3"   =>  (circshift([1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ ((4+(key-1))%12)+1 ], [ key, ((7+(key-1))%12)+1 ], ["A"]),
    "S/3"   =>  (circshift([1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], (key-1)), [ ((9+(key-1))%12)+1 ], [ key, ((5+(key-1))%12)+1 ], ["A"]),
    "D/3"   =>  (circshift([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1], (key-1)), [ ((11+(key-1))%12)+1 ], [ ((2+(key-1))%12)+1, ((7+(key-1))%12)+1 ], ["A"]),
    "T/5"   =>  (circshift([1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ ((7+(key-1))%12)+1 ], [ key, ((7+(key-1))%12)+1 ], ["W"]),
    "S/5"   =>  (circshift([1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], (key-1)), [ ((0+(key-1))%12)+1 ], [ key, ((5+(key-1))%12)+1 ], ["W"]),
    "D/5"   =>  (circshift([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1], (key-1)), [ ((2+(key-1))%12)+1 ], [ ((2+(key-1))%12)+1, ((7+(key-1))%12)+1 ], ["W"]),
    "D64"  =>  (circshift([1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ ((7+(key-1))%12)+1 ], [ ((7+(key-1))%12)+1 ], ["S"]),
    "T64"  =>  (circshift([1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], (key-1)), [ ((0+(key-1))%12)+1 ], [ key ], ["S"]),
    "T54"  =>  (circshift([1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0], (key-1)), [ ((0+(key-1))%12)+1 ], [ key ], ["S"]),
    "D54"  =>  (circshift([1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ ((7+(key-1))%12)+1 ], [ ((7+(key-1))%12)+1 ], ["S"]),
    "D7"   =>  (circshift([0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1], (key-1)), [ ((7+(key-1))%12)+1 ], [ nothing ], ["A"]),
    "D7/3" =>  (circshift([0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1], (key-1)), [ ((11+(key-1))%12)+1 ], [ nothing ], ["A"]),
    "D7/5" =>  (circshift([0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1], (key-1)), [ ((2+(key-1))%12)+1 ], [ nothing ], ["W", "O"]),
    "D7/7" =>  (circshift([0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1], (key-1)), [ ((5+(key-1))%12)+1 ], [ nothing ], ["A"]),
    "D7-1" =>  (circshift([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1], (key-1)), [ ((2+(key-1))%12)+1 ], [ ((2+(key-1))%12)+1, ((5+(key-1))%12)+1 ], ["A"]),
    "S65"  =>  (circshift([1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0], (key-1)), [ ((5+(key-1))%12)+1 ], [ nothing ], ["A"]),
    "S65/6"=>  (circshift([1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0], (key-1)), [ ((2+(key-1))%12)+1 ], [ nothing ], ["S"]),
    "S6"   =>  (circshift([0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0], (key-1)), [ ((5+(key-1))%12)+1 ], [ ((5+(key-1))%12)+1 ], ["A"]),
    "Ss"   =>  (circshift([0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0], (key-1)), [ ((2+(key-1))%12)+1 ], [ ((2+(key-1))%12)+1, ((5+(key-1))%12)+1 ], ["A"]),
    "Ts"   =>  (circshift([1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0], (key-1)), [ ((9+(key-1))%12)+1 ], [ key, ((9+(key-1))%12)+1 ], ["A"]),
    "Tm"   =>  (circshift([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1], (key-1)), [ ((4+(key-1))%12)+1 ], [ ((4+(key-1))%12)+1, ((7+(key-1))%12)+1 ], ["A"]))

elseif scale == "Minor"
#Minor  =  [1, 0, 2, 3, 0, 4, 0, 5, 6, 0, 7, 0]        
        
    functions = Dict( "T"   =>  (circshift([1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)), [1+(key-1)]),
    "S"   =>  circshift([1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], (key-1)),
    "D"   =>  circshift([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0], (key-1)),
    "T_5" =>  circshift([1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], (key-1)),
    "S_5" =>  circshift([0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], (key-1)),
    "D_5" =>  circshift([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0], (key-1)),
    "T3"  =>  circshift([1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)),
    "S3"  =>  circshift([1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], (key-1)),
    "D3"  =>  circshift([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0], (key-1)),
    "T5"  =>  circshift([1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)),
    "S5"  =>  circshift([1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], (key-1)),
    "D5"  =>  circshift([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0], (key-1)),
    "D64" =>  circshift([1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)),
    "T64" =>  circshift([1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], (key-1)),
    "T54" =>  circshift([1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0], (key-1)),
    "D54" =>  circshift([1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)),
    "D7"  =>  circshift([0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0], (key-1)),
    "D_7" =>  circshift([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0], (key-1)),
    "S65" =>  circshift([1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0], (key-1)),
    "S6"  =>  circshift([0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0], (key-1)),
    "Dm"  =>  circshift([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0], (key-1)),
    "Ts"  =>  circshift([1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0], (key-1)),
    "Tm"  =>  circshift([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0], (key-1)))
        
end
        
for k in functions
    println(k[2])
end


([1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0], [1], Nothing[nothing], ["S"])
([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0], [11], [11, 4], ["W"])
([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0], [11], [11], ["A"])
([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0], [8], [11, 8], ["A"])
([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0], [8], [11, 4], ["A"])
([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0], [6], [11, 6], ["W"])
([1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0], [1], [1, 4], ["A"])
([1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0], [1], Nothing[nothing], ["W", "O"])
([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0], [3], [11, 6], ["A"])
([1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], [1], [1, 6], ["W"])
([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0], [11], [11], ["A"])
([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0], [6], [6], ["S"])
([1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], [10], [1, 6], ["A"])
([0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], [6], [6], ["A"])
([1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0], [6], Nothing[nothing], ["A"])
([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0], [11], [11], ["S"])
([1, 0, 0, 0, 0, 1, 0, 

MethodError: MethodError: no method matching Integer(::Array{Int64,1})
Closest candidates are:
  Integer(!Matched::Integer) at boot.jl:765
  Integer(!Matched::Union{Float32, Float64}) at boot.jl:766
  Integer(!Matched::Float16) where T<:Integer at float.jl:71
  ...

# # Integer Program for Fifth enforcement

In [80]:
using JuMP, Ipopt
#model = Model(with_optimizer(GLPK.Optimizer))
#@variable()
#@constraint()
#@objective(model,,)

┌ Info: Recompiling stale cache file C:\Users\nsthtz\.julia\compiled\v1.1\JuMP\DmXqY.ji for JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1184
┌ Info: Precompiling Ipopt [b6b21f68-93f8-5de0-b562-5493be1d77c9]
└ @ Base loading.jl:1186
